In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import math
from math import floor
import imageio
import json
import pickle as pkl
import os

## Load place

In [2]:
with pd.HDFStore("place.hdf", mode="r") as store:
    print(store.keys())
    place = store.place
    users = store.users

['/place', '/users']


In [3]:
place.describe()

,ts,user_id,x_coordinate,y_coordinate,color
count,1.655990e+07,1.655990e+07,1.655990e+07,1.655990e+07,1.655990e+07
mean,2.139252e+05,2.314466e+05,4.730365e+02,5.154545e+02,5.733436e+00
std,6.689361e+04,2.229644e+05,2.882131e+02,2.996301e+02,4.437919e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.684950e+05,6.805400e+04,2.290000e+02,2.610000e+02,3.000000e+00
50%,2.249860e+05,1.619120e+05,4.590000e+02,5.080000e+02,5.000000e+00
75%,2.657800e+05,3.268200e+05,7.120000e+02,7.870000e+02,9.000000e+00
max,3.200460e+05,1.166923e+06,1.000000e+03,1.000000e+03,1.500000e+01


In [4]:
place = place[place.ts>=60845]
place.ts = place.ts - 60845

## Smoothing functions

In [70]:
def get_edits(place, _from, _to):
    canevas = place[(place.ts>=_from)&(place.ts<_to)]
    counts = canevas.shape[0]
    if counts == 0:
        return np.zeros((1000, 1000), dtype=np.float32), 0
    
    canevas = canevas.pivot_table(values="ts", index="y_coordinate", columns="x_coordinate", aggfunc=lambda x: len(x))
    canevas = canevas.reindex(index=np.arange(1001), columns=np.arange(1001))
    canevas_matrix = canevas.fillna(0).values[:-1, :-1]
    return canevas_matrix.astype(np.float32), counts

In [71]:
def get_gaussian_kernel(kernel_size=3, sigma=2, channels=1):
    # Create a x, y coordinate grid of shape (kernel_size, kernel_size, 2)
    x_coord = torch.arange(kernel_size)
    x_grid = x_coord.repeat(kernel_size).view(kernel_size, kernel_size)
    y_grid = x_grid.t()
    xy_grid = torch.stack([x_grid, y_grid], dim=-1).float()

    mean = (kernel_size - 1)/2.
    variance = sigma**2.

    # Calculate the 2-dimensional gaussian kernel which is
    # the product of two gaussian distributions for two different
    # variables (in this case called x and y)
    gaussian_kernel = (1./(2.*math.pi*variance)) *\
                      torch.exp(
                          -torch.sum((xy_grid - mean)**2., dim=-1) /\
                          (2*variance)
                      )

    # Make sure sum of values in gaussian kernel equals 1.
    gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)

    # Reshape to 2d depthwise convolutional weight
    gaussian_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size)
    gaussian_kernel = gaussian_kernel.repeat(channels, 1, 1, 1)

    gaussian_filter = nn.Conv2d(in_channels=channels, out_channels=channels,
                                kernel_size=kernel_size, groups=channels, bias=False)

    gaussian_filter.weight.data = gaussian_kernel
    gaussian_filter.weight.requires_grad = False
    
    return gaussian_filter

In [79]:
def create_canevas_tensors(place, windows_start, window_size=15*60):
    tss = []
    edits_counts = []
    batch = []
    for ts in windows_start: #range(place.ts.min(), place.ts.max()+w_size, w_size):
        tss.append(ts)
        canevas, counts = get_edits(place, ts, ts+w_size)
        #print(canevas.max())
        edits_counts.append(counts)
        batch.append(canevas)
    tensor_batch = torch.cat([torch.Tensor(m).unsqueeze(0) for m in batch])
    tensor_batch.unsqueeze_(1)
    return tss, edits_counts, tensor_batch

In [80]:
class Smoother():
    def __init__(self, kernel, sigma, max_pool, cuda=1):
        self.cuda = cuda
        self.kernel = nn.Sequential(
            nn.AvgPool2d(max_pool, max_pool),
            nn.ReflectionPad2d((kernel-1)//2),
            get_gaussian_kernel(kernel, sigma, 1))
        if cuda != None:
            self.kernel = self.kernel.cuda(cuda)
        
    def __call__(self, image_tensor):
        if self.cuda is not None:
            image_tensor = image_tensor.cuda(self.cuda)
        smoothed = self.kernel(image_tensor)
        maxs, _ = smoothed.view(smoothed.size(0), -1).max(-1)
        return smoothed.cpu().numpy().squeeze(1), maxs.cpu().numpy().squeeze()

In [87]:
def generate_levelmaps(place, smoother, ts_window, community_id=None, users_id=None, root_path="data/levelmaps"):
    windows_start = range(place.ts.min(), place.ts.max()+1, w_size)
    
    if community_id is not None:
        place = place[place.user_id.isin(users_id)]
        print(f"Community {community_id}" + "-"*10)
    tss, edits_counts, canevas_tensor = create_canevas_tensors(place, windows_start, ts_window)
    smoothed, maxs = smoother(canevas_tensor)
    
    print("Tiles generated and smoothed, saving images")
    path = f"{root_path}/{'global' if community_id is None else community_id}"
    os.makedirs(path, exist_ok=True)
    
    for i in range(len(tss)):
        imageio.imwrite(f"{path}/{i}.png", (255*smoothed[i]/max(1,smoothed[i].max())).astype(np.uint8), optimize=True)

    index = {tss[i]: {"idx":i, "max":round(float(maxs[i]),5), "counts":edits_counts[i]} for i in range(len(tss))}
    
    with open(f"{path}/index.json", "w") as f:
        json.dump({community_id: index}, f)
        print(f"Images saved to {path}")
    return index

## Apply smoothing

In [88]:
root_path = "levelmaps"
w_size = 30*60
kernel_size = 3
sigma = 1
strides = 5

smoother = Smoother(kernel_size, sigma, strides, cuda=1)

## Apply smoothing on biggest "communities"

In [89]:
n_biggest_communities = 150

In [90]:
# see place_pixels.ipynb
with open("communities.pkl", mode="rb") as f:
    communities = pkl.load(f)

In [91]:
index = {}

index["global"] = generate_levelmaps(place, smoother, w_size, None, None, root_path)

for k in sorted(communities, key=lambda x: len(communities.get(x)), reverse=True)[:n_biggest_communities+1]:
    index[k] = generate_levelmaps(place, smoother, w_size, k, communities[k], root_path)
    
with open(f"{root_path}/index.json", "w") as f:
    json.dump(index, f)


Tiles generated and smoothed, saving images
Images saved to levelmaps/global
Community 286----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/286
Community 1122----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/1122
Community 75----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/75
Community 1066----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/1066
Community 95----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/95
Community 56----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/56
Community 581----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/581
Community 73----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/73
Community 61----------
Tiles generated and smoothed, saving images
Images saved to levelmaps/61
Community 1692----------
Tiles generated and sm